In [1]:
import os
#https://www.nature.com/articles/s41586-021-03569-1 -> similar processing procedure
from tqdm import notebook
import csv
import time
import pandas as pd

In [2]:
os.getcwd()

'/scratch/hs3470/genomics'

In [3]:
os.chdir('datasets/viral/covid/') # insert the directory your covid-19 data is located at

In [21]:
df_list = [] # dataset is better read in chunks, so we create an empty list to store them and then concatenate them together
total = 2000 
chunksize = 100
pbar = notebook.tqdm(range(total)) 
for i,chunk in enumerate(pd.read_csv('GSE171524_processed_data.csv', chunksize=chunksize, low_memory=False)):
    df_list.append(chunk)
    pbar.set_description('Importing: %d' % (1 + i))
    pbar.update(chunksize)
    pbar.refresh()


  0%|          | 0/2000 [00:00<?, ?it/s]

In [23]:
processed_dataset = pd.concat(df_list).T

In [ ]:
processed_set = pd.read_csv('GSE171524_processed_data.csv')

In [7]:
processed_set = processed_set.T # in genomics data comes in form of features x samples, so we need to do some rotations

In [32]:
feature_map = pd.read_csv('GSE171524_lung_metaData.txt',sep= '\t', dtype='unicode')

In [33]:
feature_map

,NAME,biosample_id,donor_id,species,species__ontology_label,disease,disease__ontology_label,organ,organ__ontology_label,library_preparation_protocol,...,cell_type_intermediate,cell_type_fine,initial_clustering,age,sex,recorded_race,recorded_ethnicity,intubation_days,interval_death_symptoms_onset_days,pmi_h
0,TYPE,group,group,group,group,group,group,group,group,group,...,group,group,group,numeric,group,group,group,numeric,numeric,numeric
1,ATTCACTGTAACAGGC-1_1,C51ctr,C51ctr,NCBITaxon:9606,Homo sapiens,PATO:0000461,normal,UBERON:0002048,lung,EFO:0009922,...,Airway epithelial cells,Airway ciliated,18,70,female,White,not_hispanic,NaN,NaN,NaN
2,TAACTTCCAACCACGC-1_1,C51ctr,C51ctr,NCBITaxon:9606,Homo sapiens,PATO:0000461,normal,UBERON:0002048,lung,EFO:0009922,...,Macrophages,Alveolar macrophages,3,70,female,White,not_hispanic,NaN,NaN,NaN
3,TTGGGTACACGACAAG-1_1,C51ctr,C51ctr,NCBITaxon:9606,Homo sapiens,PATO:0000461,normal,UBERON:0002048,lung,EFO:0009922,...,Airway epithelial cells,Airway ciliated,18,70,female,White,not_hispanic,NaN,NaN,NaN
4,AGGCCACAGAGTCACG-1_1,C51ctr,C51ctr,NCBITaxon:9606,Homo sapiens,PATO:0000461,normal,UBERON:0002048,lung,EFO:0009922,...,Airway epithelial cells,Airway ciliated,18,70,female,White,not_hispanic,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116309,CCTAAGAAGTTGTCAC-1_27,L22cov,L22cov,NCBITaxon:9606,Homo sapiens,MONDO:0100096,COVID-19,UBERON:0002048,lung,EFO:0009922,...,Fibroblasts,Intermediate pathological FB,0,58,male,NaN,NaN,56,63,6.75
116310,CGTGAATCATGTCTAG-1_27,L22cov,L22cov,NCBITaxon:9606,Homo sapiens,MONDO:0100096,COVID-19,UBERON:0002048,lung,EFO:0009922,...,Macrophages,Transitioning MDM,9,58,male,NaN,NaN,56,63,6.75
116311,GCCGATGAGTAAACGT-1_27,L22cov,L22cov,NCBITaxon:9606,Homo sapiens,MONDO:0100096,COVID-19,UBERON:0002048,lung,EFO:0009922,...,AT2,AT2,11,58,male,NaN,NaN,56,63,6.75
116312,AGGGTGAGTATCAGGG-1_27,L22cov,L22cov,NCBITaxon:9606,Homo sapiens,MONDO:0100096,COVID-19,UBERON:0002048,lung,EFO:0009922,...,AT1,AT1,5,58,male,NaN,NaN,56,63,6.75


In [34]:
feature_map_labels = feature_map[['NAME','disease__ontology_label']]

In [35]:
label_dict={'normal':0,'COVID-19':1} # we now need to relabel the data to 0 and 1 for each sample

In [12]:
feature_map_labels = feature_map_labels.iloc[1:].replace(label_dict)
feature_map_labels

,NAME,disease__ontology_label
1,ATTCACTGTAACAGGC-1_1,0
2,TAACTTCCAACCACGC-1_1,0
3,TTGGGTACACGACAAG-1_1,0
4,AGGCCACAGAGTCACG-1_1,0
5,CACTGAAGTCGAAGCA-1_1,0
...,...,...
116309,CCTAAGAAGTTGTCAC-1_27,1
116310,CGTGAATCATGTCTAG-1_27,1
116311,GCCGATGAGTAAACGT-1_27,1
116312,AGGGTGAGTATCAGGG-1_27,1


In [16]:
processed_set

Unnamed: 0,IGKC,SFTPC,SFTPA2,IGLC2,SFTPA1,IGHG3,CEMIP,COL1A1,IGLC1,SERPINE1,...,SFTA2,AC027808.2,PFN1,FOXJ1,SLC6A4,B3GNT5,FCRL1,TNFSF15,FSTL3,RXFP1
TAGGTACCATGGCCAC-1_1,-0.172138,-0.316067,-0.269466,-0.103819,-0.269138,-0.154678,0.236594,-0.324338,-0.115634,-0.503068,...,-0.159884,-0.105492,0.080199,9.321689,-0.151069,0.311534,-0.270572,-0.06895,-0.242164,-0.469135
ATTCACTGTAACAGGC-1_1,-0.172138,-0.316067,-0.269466,-0.103819,-0.269138,-0.154678,0.332866,-0.324338,-0.115634,-0.503068,...,-0.159884,-0.105492,0.25029,8.480621,-0.151069,0.413315,-0.328076,-0.06895,-0.242164,-0.455552
TAACTTCCAACCACGC-1_1,-0.172138,-0.316067,-0.269466,-0.103819,-0.269138,-0.1559,-0.545661,-0.324338,-0.115634,-0.488057,...,-0.159884,-0.105492,3.643093,-0.08561,-0.151069,0.228501,2.955465,-0.06895,-0.242164,-0.261343
TTGGGTACACGACAAG-1_1,-0.172138,-0.316067,-0.269466,-0.103819,-0.269138,-0.154678,-0.543522,-0.324338,-0.115634,-0.503068,...,-0.159884,-0.105492,-0.168256,0.245563,-0.151069,0.661172,-0.289324,-0.06895,-0.242164,-0.108532
AGGCCACAGAGTCACG-1_1,-0.172138,-0.316067,-0.269466,-0.103819,-0.269138,-0.154678,-0.11996,-0.324338,-0.115634,-0.503068,...,-0.159884,-0.105492,0.241542,9.23362,-0.151069,1.222997,1.675507,-0.06895,-0.242164,-0.482958
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CCTAAGAAGTTGTCAC-1_27,-0.412542,-0.120986,-0.183611,-0.103639,-0.232141,-0.153952,-0.189274,2.657113,-0.412058,1.163216,...,-0.07084,-0.153621,-0.111072,-0.03652,-0.422434,-0.205699,-0.199325,-0.063466,-0.420563,-0.107651
CGTGAATCATGTCTAG-1_27,-0.171621,-0.307854,-0.253704,-0.103762,-0.264056,-0.15467,-0.261376,-0.315002,-0.115954,-0.50575,...,-0.157912,-0.128451,-0.190177,-0.085549,-0.079918,-0.040052,-0.178544,-1.286419,-0.242931,-0.11861
GCCGATGAGTAAACGT-1_27,-0.160815,0.860643,0.529802,-0.192466,0.188229,-0.151957,-0.054085,-0.328074,-0.115144,-0.500009,...,0.681025,-0.102085,-0.167807,-0.090221,-0.15562,-0.105856,-0.240867,0.11632,-0.014669,0.039856
AGGGTGAGTATCAGGG-1_27,-0.137197,-0.086531,-0.13674,-0.09044,-0.118321,-0.154018,-0.141841,-0.326726,-0.103367,-0.461035,...,-0.429645,-0.101607,-0.164949,-0.074133,-0.078381,-0.163373,-0.248969,-0.130734,0.150011,0.460347


In [14]:
new_header = processed_set.iloc[0]
processed_set = processed_set[1:]
processed_set.columns =  new_header

In [19]:
processed_set

TAGGTACCATGGCCAC-1_1,-0.172138,-0.316067,-0.269466,-0.103819,-0.269138,-0.154678,0.236594,-0.324338,-0.115634,-0.503068,...,-0.159884,-0.105492,0.080199,9.321689,-0.151069,0.311534,-0.270572,-0.068950,-0.242164,-0.469135
ATTCACTGTAACAGGC-1_1,-0.172138,-0.316067,-0.269466,-0.103819,-0.269138,-0.154678,0.332866,-0.324338,-0.115634,-0.503068,...,-0.159884,-0.105492,0.25029,8.480621,-0.151069,0.413315,-0.328076,-0.06895,-0.242164,-0.455552
TAACTTCCAACCACGC-1_1,-0.172138,-0.316067,-0.269466,-0.103819,-0.269138,-0.1559,-0.545661,-0.324338,-0.115634,-0.488057,...,-0.159884,-0.105492,3.643093,-0.08561,-0.151069,0.228501,2.955465,-0.06895,-0.242164,-0.261343
TTGGGTACACGACAAG-1_1,-0.172138,-0.316067,-0.269466,-0.103819,-0.269138,-0.154678,-0.543522,-0.324338,-0.115634,-0.503068,...,-0.159884,-0.105492,-0.168256,0.245563,-0.151069,0.661172,-0.289324,-0.06895,-0.242164,-0.108532
AGGCCACAGAGTCACG-1_1,-0.172138,-0.316067,-0.269466,-0.103819,-0.269138,-0.154678,-0.11996,-0.324338,-0.115634,-0.503068,...,-0.159884,-0.105492,0.241542,9.23362,-0.151069,1.222997,1.675507,-0.06895,-0.242164,-0.482958
CACTGAAGTCGAAGCA-1_1,-0.172138,-0.316067,-0.269466,-0.103819,-0.269138,-0.154678,0.935643,-0.324338,-0.115634,-0.503068,...,-0.159884,-0.105492,0.220908,4.954769,-0.151069,1.352841,-0.326886,-0.06895,-0.242164,-0.457076
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CCTAAGAAGTTGTCAC-1_27,-0.412542,-0.120986,-0.183611,-0.103639,-0.232141,-0.153952,-0.189274,2.657113,-0.412058,1.163216,...,-0.07084,-0.153621,-0.111072,-0.03652,-0.422434,-0.205699,-0.199325,-0.063466,-0.420563,-0.107651
CGTGAATCATGTCTAG-1_27,-0.171621,-0.307854,-0.253704,-0.103762,-0.264056,-0.15467,-0.261376,-0.315002,-0.115954,-0.50575,...,-0.157912,-0.128451,-0.190177,-0.085549,-0.079918,-0.040052,-0.178544,-1.286419,-0.242931,-0.11861
GCCGATGAGTAAACGT-1_27,-0.160815,0.860643,0.529802,-0.192466,0.188229,-0.151957,-0.054085,-0.328074,-0.115144,-0.500009,...,0.681025,-0.102085,-0.167807,-0.090221,-0.15562,-0.105856,-0.240867,0.11632,-0.014669,0.039856
AGGGTGAGTATCAGGG-1_27,-0.137197,-0.086531,-0.13674,-0.09044,-0.118321,-0.154018,-0.141841,-0.326726,-0.103367,-0.461035,...,-0.429645,-0.101607,-0.164949,-0.074133,-0.078381,-0.163373,-0.248969,-0.130734,0.150011,0.460347


In [ ]:

with open('labels.csv', 'w') as f:
    wrtr = csv.writer(f)
    count = 0
    p_bar = notebook.tqdm(range(processed_set.shape[0]))
    for index in processed_set.index[1:]: #first row is broken
        map_row = feature_map_labels.loc[feature_map_labels['NAME'] == index]
        label =  int(map_row['disease__ontology_label'])
        wrtr.writerows(zip([label]))
        p_bar.update(1)
        p_bar.refresh()

  0%|          | 0/116313 [00:00<?, ?it/s]

### Some of the sections below were made to make secondary labels for different cell types in case different classification objective was needed, for example make one cell type all label one

This is not used in the paper

In [36]:
feature_map['cell_type_intermediate'].unique()[1:]

array(['Airway epithelial cells', 'Macrophages', 'AT1', 'Smooth muscle',
       'AT2', 'Fibroblasts', 'Dendritic cells', 'Cycling NK/T cells',
       'Endothelial cells', 'Other epithelial cells', 'Mast cells',
       'Neuronal cells', 'Plasma cells', 'Monocytes', 'NK cells',
       'CD4+ T cells', 'B cells', 'Tregs', 'CD8+ T cells'], dtype=object)

In [37]:
cell_type_dict = {

'Airway epithelial cells':0,
'Macrophages':1,
'AT1':3,
'Smooth muscle':4,
'AT2':5,
'Fibroblasts':6, 
'Dendritic cells':7, 
'Cycling NK/T cells':8,
'Endothelial cells':9, 
'Other epithelial cells':10, 
'Mast cells':11,
'Neuronal cells':12, 
'Plasma cells':13, 
'Monocytes':14, 
'NK cells':15,
'CD4+ T cells':16, 
'B cells':17, 
'Tregs':18, 
'CD8+ T cells':19
}

In [38]:
feature_map_labels2= feature_map[['NAME','disease__ontology_label','cell_type_intermediate']]

In [39]:
feature_map_labels2

,NAME,disease__ontology_label,cell_type_intermediate
0,TYPE,group,group
1,ATTCACTGTAACAGGC-1_1,normal,Airway epithelial cells
2,TAACTTCCAACCACGC-1_1,normal,Macrophages
3,TTGGGTACACGACAAG-1_1,normal,Airway epithelial cells
4,AGGCCACAGAGTCACG-1_1,normal,Airway epithelial cells
...,...,...,...
116309,CCTAAGAAGTTGTCAC-1_27,COVID-19,Fibroblasts
116310,CGTGAATCATGTCTAG-1_27,COVID-19,Macrophages
116311,GCCGATGAGTAAACGT-1_27,COVID-19,AT2
116312,AGGGTGAGTATCAGGG-1_27,COVID-19,AT1


In [40]:
feature_map_labels2 = feature_map_labels2.iloc[1:].replace(label_dict)
feature_map_labels2 = feature_map_labels2.replace(cell_type_dict)
feature_map_labels2

,NAME,disease__ontology_label,cell_type_intermediate
1,ATTCACTGTAACAGGC-1_1,0,0
2,TAACTTCCAACCACGC-1_1,0,1
3,TTGGGTACACGACAAG-1_1,0,0
4,AGGCCACAGAGTCACG-1_1,0,0
5,CACTGAAGTCGAAGCA-1_1,0,0
...,...,...,...
116309,CCTAAGAAGTTGTCAC-1_27,1,6
116310,CGTGAATCATGTCTAG-1_27,1,1
116311,GCCGATGAGTAAACGT-1_27,1,5
116312,AGGGTGAGTATCAGGG-1_27,1,3


In [ ]:
with open('labels2.csv', 'w') as f:
    wrtr = csv.writer(f)
    count = 0
    wrtr.writerows(zip(['label'],['secondary_cell_label']))
    p_bar = notebook.tqdm(range(processed_set.shape[0]))
    for index in processed_set.index[:]: #first row is broken
        map_row = feature_map_labels2.loc[feature_map_labels2['NAME'] == index]
        label =  int(map_row['disease__ontology_label'])
        secondary_cell_label = int(map_row['cell_type_intermediate'])
        wrtr.writerows(zip([label],[secondary_cell_label]))
        p_bar.update(1)
        p_bar.refresh()

  0%|          | 0/116313 [00:00<?, ?it/s]

In [61]:
labels2 = pd.read_csv('labels2.csv')

In [71]:
labels2

,label,secondary_cell_label
0,0,1
1,0,0
2,0,0
3,0,0
4,0,3
...,...,...
116307,1,6
116308,1,1
116309,1,5
116310,1,3


In [30]:
new_header = processed_dataset.iloc[0]
processed_dataset = processed_dataset[1:]
processed_dataset.columns =  new_header

In [42]:
d = dict.fromkeys(['disease__ontology_label','cell_type_intermediate'], 0)
temp_df = pd.DataFrame(d, index=processed_dataset.index)

In [47]:
temp_df = temp_df.iloc[1:]
temp_df

,disease__ontology_label,cell_type_intermediate
ATTCACTGTAACAGGC-1_1,0,0
TAACTTCCAACCACGC-1_1,0,0
TTGGGTACACGACAAG-1_1,0,0
AGGCCACAGAGTCACG-1_1,0,0
CACTGAAGTCGAAGCA-1_1,0,0
...,...,...
CCTAAGAAGTTGTCAC-1_27,0,0
CGTGAATCATGTCTAG-1_27,0,0
GCCGATGAGTAAACGT-1_27,0,0
AGGGTGAGTATCAGGG-1_27,0,0


In [48]:
processed_dataset = processed_dataset.iloc[1:]

In [57]:
feature_map_labels2 =  feature_map_labels2.set_index('NAME')

In [58]:
feature_map_labels2

,disease__ontology_label,cell_type_intermediate
NAME,,
ATTCACTGTAACAGGC-1_1,0,0
TAACTTCCAACCACGC-1_1,0,1
TTGGGTACACGACAAG-1_1,0,0
AGGCCACAGAGTCACG-1_1,0,0
CACTGAAGTCGAAGCA-1_1,0,0
...,...,...
CCTAAGAAGTTGTCAC-1_27,1,6
CGTGAATCATGTCTAG-1_27,1,1
GCCGATGAGTAAACGT-1_27,1,5


In [59]:
pd.concat([processed_dataset,feature_map_labels2[['disease__ontology_label','cell_type_intermediate']]],axis=1)

,IGKC,SFTPC,SFTPA2,IGLC2,SFTPA1,IGHG3,CEMIP,COL1A1,IGLC1,SERPINE1,...,PFN1,FOXJ1,SLC6A4,B3GNT5,FCRL1,TNFSF15,FSTL3,RXFP1,disease__ontology_label,cell_type_intermediate
ATTCACTGTAACAGGC-1_1,-0.172138,-0.316067,-0.269466,-0.103819,-0.269138,-0.154678,0.332866,-0.324338,-0.115634,-0.503068,...,0.25029,8.480621,-0.151069,0.413315,-0.328076,-0.06895,-0.242164,-0.455552,0,0
TAACTTCCAACCACGC-1_1,-0.172138,-0.316067,-0.269466,-0.103819,-0.269138,-0.1559,-0.545661,-0.324338,-0.115634,-0.488057,...,3.643093,-0.08561,-0.151069,0.228501,2.955465,-0.06895,-0.242164,-0.261343,0,1
TTGGGTACACGACAAG-1_1,-0.172138,-0.316067,-0.269466,-0.103819,-0.269138,-0.154678,-0.543522,-0.324338,-0.115634,-0.503068,...,-0.168256,0.245563,-0.151069,0.661172,-0.289324,-0.06895,-0.242164,-0.108532,0,0
AGGCCACAGAGTCACG-1_1,-0.172138,-0.316067,-0.269466,-0.103819,-0.269138,-0.154678,-0.11996,-0.324338,-0.115634,-0.503068,...,0.241542,9.23362,-0.151069,1.222997,1.675507,-0.06895,-0.242164,-0.482958,0,0
CACTGAAGTCGAAGCA-1_1,-0.172138,-0.316067,-0.269466,-0.103819,-0.269138,-0.154678,0.935643,-0.324338,-0.115634,-0.503068,...,0.220908,4.954769,-0.151069,1.352841,-0.326886,-0.06895,-0.242164,-0.457076,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CCTAAGAAGTTGTCAC-1_27,-0.412542,-0.120986,-0.183611,-0.103639,-0.232141,-0.153952,-0.189274,2.657113,-0.412058,1.163216,...,-0.111072,-0.03652,-0.422434,-0.205699,-0.199325,-0.063466,-0.420563,-0.107651,1,6
CGTGAATCATGTCTAG-1_27,-0.171621,-0.307854,-0.253704,-0.103762,-0.264056,-0.15467,-0.261376,-0.315002,-0.115954,-0.50575,...,-0.190177,-0.085549,-0.079918,-0.040052,-0.178544,-1.286419,-0.242931,-0.11861,1,1
GCCGATGAGTAAACGT-1_27,-0.160815,0.860643,0.529802,-0.192466,0.188229,-0.151957,-0.054085,-0.328074,-0.115144,-0.500009,...,-0.167807,-0.090221,-0.15562,-0.105856,-0.240867,0.11632,-0.014669,0.039856,1,5
AGGGTGAGTATCAGGG-1_27,-0.137197,-0.086531,-0.13674,-0.09044,-0.118321,-0.154018,-0.141841,-0.326726,-0.103367,-0.461035,...,-0.164949,-0.074133,-0.078381,-0.163373,-0.248969,-0.130734,0.150011,0.460347,1,3


In [55]:
feature_map_labels2[['disease__ontology_label','cell_type_intermediate']]

,disease__ontology_label,cell_type_intermediate
1,0,0
2,0,1
3,0,0
4,0,0
5,0,0
...,...,...
116309,1,6
116310,1,1
116311,1,5
116312,1,3


### Here we form final dataset for the models, disease__ontology_label is our true label for the task

In [61]:
final_df = pd.merge(processed_dataset, feature_map_labels2[['disease__ontology_label','cell_type_intermediate']], left_index=True, right_index=True)

In [63]:
final_df.to_csv('covid_data.csv')

In [ ]:
final_df